<a href="https://colab.research.google.com/github/LoQiseaking69/SM2/blob/main/SephMnotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports
This section includes all necessary library imports. Ensure that all the libraries used in the notebook are imported here.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers
from collections import deque
import gym
import random
import logging

# Model Build
Detailed description of the model building process. This section should include information about the architecture, layers used, and any custom components.

In [ ]:
class ReplayBuffer:
    """Replay Buffer for storing transitions."""
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)

    def store_transition(self, transition):
        self.buffer.append(transition)

    def sample_buffer(self, batch_size):
        if len(self.buffer) < batch_size:
            return None
        samples = np.array(random.sample(self.buffer, batch_size), dtype=object)
        return [np.stack(samples[:, i]) for i in range(samples.shape[1])]

class RBMLayer(layers.Layer):
    """Restricted Boltzmann Machine Layer."""
    def __init__(self, num_hidden_units):
        super(RBMLayer, self).__init__()
        self.num_hidden_units = num_hidden_units

    def build(self, input_shape):
        if len(input_shape) != 2:
            raise ValueError("RBMLayer expects input shape of length 2")
        self.rbm_weights = self.add_weight(shape=(input_shape[-1], self.num_hidden_units),
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.biases = self.add_weight(shape=(self.num_hidden_units,),
                                      initializer='zeros',
                                      trainable=True)

    def call(self, inputs):
        activation = tf.matmul(inputs, self.rbm_weights) + self.biases
        return tf.nn.sigmoid(activation)

class QLearningLayer(layers.Layer):
    """Q-Learning Layer for reinforcement learning."""
    def __init__(self, action_space_size, learning_rate=0.001, gamma=0.99, epsilon=0.1):
        super(QLearningLayer, self).__init__()
        self.action_space_size = action_space_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = 0.995
        self.min_epsilon = 0.01
        self.replay_buffer = ReplayBuffer(100000)

    def build(self, input_shape):
        self.q_network = models.Sequential([
            layers.Dense(256, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.01)),
            layers.Dense(self.action_space_size, kernel_initializer='glorot_uniform')
        ])
        self.q_network.compile(optimizer=optimizers.Adam(learning_rate=self.learning_rate), loss='mse')
        self.target_q_network = models.clone_model(self.q_network)

    def call(self, state):
        return self.q_network(state)

    def update(self, batch_size):
        data = self.replay_buffer.sample_buffer(batch_size)
        if data is None:
            return
        states, actions, rewards, next_states, dones = data
        target_q_values = rewards + (1 - dones) * self.gamma * np.max(self.target_q_network.predict(next_states), axis=1)
        with tf.GradientTape() as tape:
            q_values = tf.reduce_sum(self.q_network(states) * tf.one_hot(actions, self.action_space_size), axis=1)
            loss = tf.reduce_mean(tf.square(target_q_values - q_values))
        grads = tape.gradient(loss, self.q_network.trainable_variables)
        self.q_network.optimizer.apply_gradients(zip(grads, self.q_network.trainable_variables))
        if self.epsilon > self.min_epsilon:
            self.epsilon *= self.epsilon_decay
        if self.buffer_index % 1000 == 0:
            self.target_q_network.set_weights(self.q_network.get_weights())

    def store_transition(self, state, action, reward, next_state, done):
        self.replay_buffer.store_transition((state, action, reward, next_state, done))

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.action_space_size)
        else:
            q_values = self.q_network.predict(state[np.newaxis, :])
            return np.argmax(q_values[0])

def positional_encoding(seq_length, d_model):
    """Positional encoding for sequence data."""
    position = np.arange(seq_length)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    pos_encoding = np.zeros((seq_length, d_model))
    pos_encoding[:, 0::2] = np.sin(position * div_term)
    pos_encoding[:, 1::2] = np.cos(position * div_term)
    return tf.constant(pos_encoding, dtype=tf.float32)

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    """Transformer encoder layer."""
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Dense(ff_dim, activation='relu', kernel_initializer='he_uniform')(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1], kernel_initializer='glorot_uniform')(x)
    return x + res

def create_neural_network_model(seq_length, d_model, num_hidden_units, action_space_size):
    """Create a neural network model integrating various layers."""
    input_layer = layers.Input(shape=(seq_length, d_model))
    x = positional_encoding(seq_length, d_model)
    x = x + input_layer
    x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=256)
    x_lstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True, kernel_initializer='glorot_uniform'))(x)
    x_conv = layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_uniform')(x_lstm)

    # Reshaping the output from Conv1D to match the 2D input expectation of RBMLayer
    x_flatten = layers.Flatten()(x_conv)
    x_rbm = RBMLayer(num_hidden_units)(x_flatten)

    q_learning_layer = QLearningLayer(action_space_size)(x_rbm)

    model = models.Model(inputs=input_layer, outputs=q_learning_layer)
    return model

# Train
This section is dedicated to the training process of the model. It should include details about the training loop, optimization process, and any data preprocessing.

In [ ]:
# Initialize the model with appropriate parameters
seq_length = 24  # Example value, set it according to your environment
d_model = 16     # Example value, set it according to your environment
num_hidden_units = 50  # Example value
action_space_size = 4  # Example value, set it according to the environment
model = create_neural_network_model(seq_length, d_model, num_hidden_units, action_space_size)

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Training Function
def train_model_in_bipedalwalker(env_name, model, num_episodes, batch_size=64):
    try:
        env = gym.make(env_name)
    except gym.error.Error as e:
        logger.error(f"Error creating environment {env_name}: {e}")
        return

    for episode in range(num_episodes):
        try:
            state = env.reset()
            state = np.array(state).reshape(1, -1)  # Reshape for model input
            done = False
            total_reward = 0

            while not done:
                action = model.choose_action(state)
                next_state, reward, done, _ = env.step(action)
                next_state = np.array(next_state).reshape(1, -1)  # Reshape for model input
                model.store_transition(state, action, reward, next_state, done)
                model.update(batch_size)
                state = next_state
                total_reward += reward

            logger.info(f'Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}')
        except Exception as e:
            logger.error(f"An error occurred in episode {episode + 1}: {e}")

    env.close()

    # Save the trained model in the current notebook directory
    notebook_dir = os.getcwd()
    save_path = os.path.join(notebook_dir, 'trained_model')
    try:
        model.save(save_path, save_format='tf')
        logger.info(f"Model saved successfully at {save_path}")
    except Exception as e:
        logger.error(f"An error occurred while saving the model: {e}")

# Example usage
env_name = 'BipedalWalker-v3'
num_episodes = 1000
train_model_in_bipedalwalker(env_name, model, num_episodes)

ERROR:__main__:Error creating environment BipedalWalker-v3: box2D is not installed, run `pip install gym[box2d]`


# Eval
Description of the evaluation process. This section should cover how the model is evaluated, including metrics used, test datasets, and interpretation of the results.

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define or import your custom layers here (RBMLayer, QLearningLayer)
# Ensure these definitions are the same as in your model creation

def load_model(model_path):
    """
    Load the saved model from a specified path.
    """
    try:
        custom_objects = {'RBMLayer': RBMLayer, 'QLearningLayer': QLearningLayer}
        model = models.load_model(model_path, custom_objects=custom_objects)
        logger.info("Model loaded successfully.")
        return model
    except Exception as e:
        logger.error(f"Error loading model: {e}")
        raise

def evaluate_model(model, env_name, num_episodes):
    """
    Evaluate the model on the environment over a number of episodes.
    """
    env = gym.make(env_name)
    total_rewards = []

    for episode in range(num_episodes):
        state = env.reset()
        state = np.array(state).reshape(1, -1)  # Reshape for model input
        done = False
        total_reward = 0

        while not done:
            action = model.choose_action(state)
            next_state, reward, done, _ = env.step(action)
            next_state = np.array(next_state).reshape(1, -1)  # Reshape for model input
            state = next_state
            total_reward += reward

        total_rewards.append(total_reward)
        logger.info(f'Episode: {episode + 1}, Total Reward: {total_reward}')

    env.close()
    avg_reward = np.mean(total_rewards)
    std_reward = np.std(total_rewards)
    logger.info(f'Average Reward over {num_episodes} episodes: {avg_reward}')
    logger.info(f'Standard Deviation of Reward: {std_reward}')
    return avg_reward, std_reward

# Example usage
model_path = 'path_to_your_saved_model_directory/trained_model'  # Adjust this path as needed
loaded_model = load_model(model_path)

env_name = 'BipedalWalker-v3'
num_episodes = 100  # Number of episodes for evaluation

# Evaluate and print model performance
avg_reward, std_reward = evaluate_model(loaded_model, env_name, num_episodes)
print(f'Average Reward: {avg_reward}, Standard Deviation of Reward: {std_reward}')